# Project - 5
## Vehicle Detection and Tracking

**Goals of this project are: ** 



- Generate/Import classified images corresponding to vehicle and non vehicle classes
- Generate necessary and required features from the image to classy objects in the image as cars and noncars.
- Train a classifier to classify objects in the image as cars and noncars.
- Define methods to gain confidence in the detections and segregating false detections.
- Write method to apply the vehicle detection pipeline on the project video

In this project writeup, i will be fulfilling each rubrick point individually.

**Explain how (and identify where in your code) you extracted HOG features from the training images. Explain how you settled on your final choice of HOG parameters.**

HOG stands for Histogram of Oriented Gradients. It is a technique to represent shape of the object or objects present in the image. As, our task here is to classify vehicles from rest of the universe, we train our model with some number of car images and roughly equal number of noncar images. These numbers are roughly equal to prevent model from getting biased towards a particular output.  
The HOG function by skimage, gives some parameters that can be tweaked to get required feature out of the image. The parameters are: Orientation, Pixels per cell, cell per block, etc.  
In my model, i have used the following values of these parameters.
- Orientation = 11
- Pixels per cell = 16
- Cell per block = 2  

These values are based on the exploration done during the quizes in the lesson.  
Hog function takes a single channel image as its input. So, we should use the channel that best represents the shape of the objects in it. As channels of the image are not classified on the basis of shape properties, our best option is to use all the three channels and concatenate hog features of each channel. And this is what i have done in the project.  
Following image shows an image and corresponding HOG feature image.

![HogDemo](OutputImages/HogDemo.png)

** Hog feature extraction function: **  
```python
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    """
    Returns Hog feature vector.
    Parameters are: Orientations bins, Pixels per cell and Cells per block
    """
    # Call with two outputs if vis==True
    
    if vis == True:
        features, hog_image = hog(img, orientations=orient,
                                pixels_per_cell=(pix_per_cell, pix_per_cell),
                                cells_per_block=(cell_per_block, cell_per_block), 
                                transform_sqrt=False, 
                                visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

```

**Describe how (and identify where in your code) you trained a classifier using your selected HOG features. **

Along with HOG features, i have also used color histogram features. Color histogram, contains the distribution of clors in the image. Number of bins used for color histogram is 32 for each channel so, there are 32 x 3 = 96 features corresponding to color histogram.  
As suggested by the instructors in the lesson, i have used SVC classifier. I also tested for binary tree, which was giving almost same accuracy as that of SVC but was taking more time to train. There is one more factor for not using binary tree classifier, that i will discuss in later section.  
Raw pixels are not used as features because, we have both color and shape features with us and using extra 3072 (32x32x3) features will result in redundancy.  
As linear SVC was giving far better accuracy than nonlinear SVC, former is used.  

Following code snippet shows training of classifier.
```python
X = np.vstack((car_features, notcar_features)).astype(np.float64)

y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

rand_state = np.random.randint(0, 100)

X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

svc = LinearSVC()
t = time.time()
svc.fit(X_train, y_train)
t2 = time.time()

```

**Describe how (and identify where in your code) you implemented a sliding window search. How did you decide what scales to search and how much to overlap windows?**

Our aim while defining the sliding window search is that, we have to accomodate the shrinking of objects as they go farther away. This can be done by varying the size of the search window with y axis of the image. This is what i tried to do in the following scheme. In this scheme, max size and min size of the window are given, and it linearly varies with y axis. The code snippet, along with generated windows are shown below.

```python
def generate_templates(img, x_overlap = 0.5, y_overlap = 0.7, max_size = 150, min_size = 32):
    """
    Function to generate templates with size varying with 'Y' 
    NOT USED
    """
    image = np.copy(img)
    h,w = image[:,:,-1].shape
    # y_start and y_stop are limits that define which parts of the image is to be searched.
    y_stop = h//3; y_start = h-50;
    size = max_size # Set the initial template size to max size
    template_per_y = np.int((w/(max_size*(1 - x_overlap)))) # Number of templates at each y is same.
    x_start = 0; # Templates start with an x offset that is initially zero
    y_leftbottom = 0.0; x_leftbottom = 0; # y_leftbottom starts with zero and goes upto y_max. It is later transformed into image coordinates.
    templates = [] # Create an empty list to save the templates
    while y_leftbottom < y_stop :
        for i in range(template_per_y):
            x_leftbottom = x_start + (i*size*(1-x_overlap))
            x_topright = x_leftbottom + size            
            startx = np.int(x_leftbottom)
            starty = np.int(y_start - y_leftbottom)
            endx = np.int(x_topright)
            endy = np.int(starty - size)
            templates.append(((startx, starty), (endx, endy)))
        y_leftbottom += np.int((1-y_overlap)*size) # shift y for next row
        x_start += w//25 # Update x offset
        size = max_size - y_leftbottom*((max_size - min_size)/y_stop) # Update template size(linear equation is used)
        x_overlap = 0.5*(1 - y_leftbottom/y_stop) # x_overlap decreases with y
    return templates
```

![Mytemplates](OutputImages/Mytemplates.png)

However, we want that there should be a large number of true positive templates around the cars in our image because these templates will result in a good heatmap of positive detections making removal of false positives easy. And that condition is not achieved with the above mentioned template scheme. So, instead of that, I have used the scheme demonstrated in the lesson. The lesson scheme reults in following templates.

![HogTemplates](OutputImages/HogTemplates.png)

**Show some examples of test images to demonstrate how your pipeline is working. How did you optimize the performance of your classifier?**

Following is the output of the vehicle detection pipeline on the test images.

![PipeLineoutput](OutputImages/PipelineOutputTestImages.png)

**Describe how (and identify where in your code) you implemented some kind of filter for false positives and some method for combining overlapping bounding boxes.**

The classifier is detecting various false positives. Even on trying with multiple color channels, accuracy didnot improve much. So, we have to take a strategy other than color channel.  
What i have done is, i have run the vehicle detection pipeline on the same image multiple times, with different scale values. What this does is, it scales the image first and then searches for vehicle. This scheme is beneficial in two ways, firstly, it incorporates shrinking of vehicles with distance by multiple scaling of the image. Secondly, with more and more numbers of rectangle detections, we get stronger heat for true detections and weaker heat for false positives. This way, we can remove the false positives easily. Though we are left with a few false positives, the pipeline detects vehicles for almost complete video.

Shown below is the detections for testimage corresponding to different scales we have used.

![MultipleScales](OutputImages/MultipleScales.png)

And their corresponding heatmaps

![Heatmap](OutputImages/MultipleHeats.png)

For combining overlapping boxes, we have used the above presented heatmap technique along with scipy.ndimage.measurements.label() on the integrated heatmap as shown in the following code snippet.

```python
bboxes = find_cars(image,  svc, X_scaler,scale = 2.6)    
    for box in bboxes:
        rectangles.append(box)
        
    heat = addheat(image[:,:,0].shape, rectangles)    
    heat = apply_threshold(heat, threshold = 5)
    heatmap = np.clip(heat, 0, 255)
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    
    return draw_img
```

**Briefly discuss any problems / issues you faced in your implementation of this project. Where will your pipeline likely fail? What could you do to make it more robust?**

In this project i have used Color feature and shape features to classify vehicles from non vehicles in the provided image. The main challenge i faced in this project was removing false detections. Though they were not completely eradicated but were significantly reduced after performing hard mining.    
This scheme is limited right now to the vehicles that are travelling in the same direction as our vehicles. Because all the test images provided were belonging to the back side view of vehicles. A vehicle coming from front will be detected as not a car. So, more data should be collected to train our classifier to detect vehicles coming from front as well.  
As we saw, even after taking all the precautions, there were multiple false positives, that means, our classifier training technique needs to be improved. One thing that we can do is to train multiple classifiers and make detection true only when both classifiers give positive response. This way false positives can be reduced significantly.  

